### --- Step 1: Imports ---

In [1]:
from functions.llm_chain import generate_response_openai

: 

In [1]:
import hopsworks
import exclude.key
from openai import OpenAI
import os
import getpass
import joblib

project = hopsworks.login()
fs = project.get_feature_store() 

# Hopsworks
HOPSWORKS_API_KEY = exclude.key.HOPSWORKS_API_KEY
FEATURE_GROUP_NAME = "bars_near_london_bridge"
FEATURE_VIEW_NAME = "busyness_feature_view"
FEATURE_GROUP_VERSION = 3
FEATURE_VIEW_VERSION = 4

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/148
Connected. Call `.close()` to terminate connection gracefully.


## -- Step 2 Retrieve Feature View -- 

In [ ]:
# Retrieve the 'air_quality_fv' feature view
feature_view = fs.get_feature_view(
    name='FEATURE_VIEW_NAME',
    version=FEATURE_VIEW_VERSION,
)

# Initialize batch scoring
feature_view.init_batch_scoring(1)

In [ ]:
# Retrieve the model registry
mr = project.get_model_registry()

# Retrieve the 'air_quality_xgboost_model' from the model registry
retrieved_model = mr.get_model(
    name="xgboost_busyness",
    version=1,
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

# Load the XGBoost regressor model and label encoder from the saved model directory
model_busyness = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
encoder = joblib.load(saved_model_dir + "/label_encoder.pkl")

# Display the retrieved XGBoost regressor model
model_busyness


## -- Step 3 Ask the Question! -- 

In [ ]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass.getpass('🔑 Enter your OpenAI API key: ')

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
)

In [ ]:
QUESTION = "What will be the best bar to go in an hour from now next to London Bridge?"

response = generate_response_openai(   
    QUESTION,
    feature_view,
    model_busyness,
    encoder,
    client,
    verbose=True,
)
print(response)